<a href="https://colab.research.google.com/github/owilli38/DSBA-6162/blob/main/DSBA6162_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My Drive/ColabData
%cd /content/drive/My Drive/ColabData

/content/drive/My Drive/ColabData


In [ ]:
import os
extracted_dir = 'MovieReviews_extracted/MovieReviews'

In [ ]:
all_files = os.listdir(extracted_dir)
text_files = [f for f in all_files if f.endswith('.txt')]
text_file_paths = [os.path.join(extracted_dir, f) for f in text_files]

In [ ]:
data_list = []
for file_path in text_file_paths:
    with open(file_path, 'r', encoding='latin-1') as f:
        content = f.read()
    filename = os.path.basename(file_path)
    data_list.append({'filename': filename, 'content': content})

In [ ]:
import pandas as pd
df = pd.DataFrame(data_list)
display(df.head())

,filename,content
0,16748.txt,"DENNIS SCHWARTZ ""Movie Reviews and Poetry""\nUN..."
1,17108.txt,"A brilliant, witty mock documentary of Jean Se..."
2,17109.txt,NOSTALGHIA (director: Andrei Tarkovsky; cast: ...
3,17110.txt,PAYBACK (director: Brian Helgeland; cast:(Port...
4,17111.txt,WAKING NED DEVINE (director: Kirk Jones (III);...


In [ ]:
display(df.head())

,filename,content
0,16748.txt,"DENNIS SCHWARTZ ""Movie Reviews and Poetry""\nUN..."
1,17108.txt,"A brilliant, witty mock documentary of Jean Se..."
2,17109.txt,NOSTALGHIA (director: Andrei Tarkovsky; cast: ...
3,17110.txt,PAYBACK (director: Brian Helgeland; cast:(Port...
4,17111.txt,WAKING NED DEVINE (director: Kirk Jones (III);...


In [ ]:
df

,filename,content
0,16748.txt,"DENNIS SCHWARTZ ""Movie Reviews and Poetry""\nUN..."
1,17108.txt,"A brilliant, witty mock documentary of Jean Se..."
2,17109.txt,NOSTALGHIA (director: Andrei Tarkovsky; cast: ...
3,17110.txt,PAYBACK (director: Brian Helgeland; cast:(Port...
4,17111.txt,WAKING NED DEVINE (director: Kirk Jones (III);...
...,...,...
175,2101.txt,"Well, either CALENDAR GIRL is a blatant attemp..."
176,2108.txt,"For those who live in and around Philadelphia,..."
177,2110.txt,"Starring: Suzy Amis, Bo Hopkins, Ian McKellen,..."
178,2113.txt,"Starring: Jesse Bradford, Jeroen Krabbe, Lisa ..."


In [ ]:
data_text = df[['content']]

In [ ]:
#add a column to data_text for the row index
data_text['index'] = data_text.index

In [ ]:
len(data_text)

180

In [ ]:
data_sample = data_text[data_text['index'] == 101]
data_sample

,content,index
101,"In 1963, Frank Horrigan was among the ""best an...",101


In [ ]:
doc_sample = data_sample.values[0][0]

In [ ]:
doc_sample

'In 1963, Frank Horrigan was among the "best and brightest" of the Secret Service, the personal choice of President Kennedy. On November 22 in Dallas, an afternoon that Horrigan will never forget, he became one of the few living agents to lose a president. Now, decades later and only months before a presidential election (perhaps 1992 or 1996--we\'re never told exactly), Mitch Leary (John Malkovich) is stalking the President. Believing there to be an intangible bond between himself and Horrigan (he says they were both betrayed by a government they had once loved), Leary makes contact with the Secret Service agent to discuss his intentions. After his contact with the potential killer, Horrigan acts quickly to get himself posted to protection duty. This time, he has no intention of failing and believes that, given the opportunity, he will take the bullet. But Leary has everything on his side--time, opportunity, and a group of pig-headed behind-the-scenes men at the White House who refuse

In [ ]:
data_sample = data_text[data_text['index'] == 101]
data_sample

,content,index
101,"In 1963, Frank Horrigan was among the ""best an...",101


In [ ]:
!pip install gensim

In [ ]:
#We will perform the following steps:
 #Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation. Done by "gensim.utils.simple_preprocess()"
 #Words that have fewer than 3 characters are removed.
 #All stopwords are removed.
 #Words are lemmatized — verbs in third person are changed to first person and verbs in past and future tenses are changed into present tense.
 #Words are stemmed — words are reduced to their root form.


#import relvant packages for conduct topic modeling analysis
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

from nltk.stem import *
stemmer = PorterStemmer()

In [ ]:
len(gensim.parsing.preprocessing.STOPWORDS)

337

In [ ]:
#a function to perform lemmatize and stem preprocessing steps on the data set. Without contextual information the lemmatiser’s not able to distinguish between nouns and verbs. In this case, "pos = 'v'" means treat the words as verbs.

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [ ]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
#remove na values from the column 'headline_text'
data_text = data_text.dropna(subset=['content'])

In [ ]:
#select a document to preview after preprocessing
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\npreprocessed document: ')
print(preprocess(doc_sample))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['In', '1963,', 'Frank', 'Horrigan', 'was', 'among', 'the', '"best', 'and', 'brightest"', 'of', 'the', 'Secret', 'Service,', 'the', 'personal', 'choice', 'of', 'President', 'Kennedy.', 'On', 'November', '22', 'in', 'Dallas,', 'an', 'afternoon', 'that', 'Horrigan', 'will', 'never', 'forget,', 'he', 'became', 'one', 'of', 'the', 'few', 'living', 'agents', 'to', 'lose', 'a', 'president.', 'Now,', 'decades', 'later', 'and', 'only', 'months', 'before', 'a', 'presidential', 'election', '(perhaps', '1992', 'or', "1996--we're", 'never', 'told', 'exactly),', 'Mitch', 'Leary', '(John', 'Malkovich)', 'is', 'stalking', 'the', 'President.', 'Believing', 'there', 'to', 'be', 'an', 'intangible', 'bond', 'between', 'himself', 'and', 'Horrigan', '(he', 'says', 'they', 'were', 'both', 'betrayed', 'by', 'a', 'government', 'they', 'had', 'once', 'loved),', 'Leary', 'makes', 'contact', 'with', 'the', 'Secret', 'Service', 'agent', 'to', 'discuss', 'his', 'intentions.', 'After', 'his', 'contact', 'with', 'th

In [ ]:
#preprocess'headline_text', save the results as 'processed_docs'
processed_docs = data_text['content'].map(preprocess)
processed_docs[:10]

,content
0,"[denni, schwartz, movi, review, poetri, unmak,..."
1,"[brilliant, witti, mock, documentari, jean, se..."
2,"[nostalghia, director, andrei, tarkovski, cast..."
3,"[payback, director, brian, helgeland, cast, po..."
4,"[wake, devin, director, kirk, jone, cast, bann..."
5,"[happi, director, todd, solondz, cast, dylan, ..."
6,"[leon, morin, priest, director, jean, pierr, m..."
7,"[bich, director, claud, chabrol, cast, jean, l..."
8,"[funni, game, director, michael, hanek, cast, ..."
9,"[gun, director, john, sayl, cast, federico, lu..."


In [ ]:
#create a dictionary from ‘processed_docs’ containing the number of times a word appears in the document set
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
#Filter out tokens that appear in
 #less than 15 documents (absolute number) or
 #more than 0.5 documents (fraction of total corpus size, not absolute number).
 #after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
#For each document we created a dictionary reporting how many
#words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[101]

[(1, 1),
 (19, 1),
 (21, 1),
 (25, 1),
 (29, 1),
 (33, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 2),
 (48, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (62, 1),
 (68, 1),
 (71, 1),
 (74, 1),
 (78, 1),
 (82, 1),
 (84, 1),
 (92, 1),
 (100, 1),
 (104, 1),
 (105, 1),
 (106, 3),
 (107, 3),
 (108, 1),
 (113, 2),
 (119, 1),
 (123, 1),
 (125, 1),
 (128, 1),
 (133, 1),
 (139, 1),
 (154, 1),
 (156, 1),
 (160, 1),
 (161, 2),
 (163, 3),
 (164, 4),
 (176, 1),
 (179, 1),
 (187, 1),
 (195, 1),
 (206, 4),
 (219, 1),
 (222, 1),
 (224, 1),
 (231, 9),
 (238, 1),
 (239, 1),
 (245, 1),
 (251, 1),
 (257, 2),
 (276, 1),
 (280, 1),
 (288, 1),
 (289, 1),
 (300, 3),
 (303, 1),
 (307, 1),
 (316, 1),
 (323, 1),
 (324, 3),
 (341, 1),
 (343, 1),
 (345, 1),
 (359, 1),
 (370, 1),
 (372, 1),
 (373, 2),
 (377, 2),
 (379, 2),
 (383, 1),
 (384, 1),
 (389, 1),
 (393, 1),
 (395, 1),
 (397, 3),
 (398, 1),
 (401, 1),
 (406, 2),
 (408, 1),
 (413, 1),
 (416, 1),
 (423, 1),
 (426, 1),
 (432, 2),
 (43

In [ ]:
for i in range(len(bow_corpus[101])):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus[101][i][0], dictionary[bow_corpus[101][i][0]], bow_corpus[101][i][1]))

Word 1 ("actor") appears 1 time.
Word 19 ("date") appears 1 time.
Word 21 ("deal") appears 1 time.
Word 25 ("excit") appears 1 time.
Word 29 ("fail") appears 1 time.
Word 33 ("femal") appears 1 time.
Word 40 ("hope") appears 1 time.
Word 41 ("human") appears 1 time.
Word 42 ("imag") appears 1 time.
Word 43 ("includ") appears 1 time.
Word 44 ("interest") appears 1 time.
Word 45 ("intrigu") appears 1 time.
Word 46 ("ladi") appears 1 time.
Word 47 ("littl") appears 2 time.
Word 48 ("lose") appears 1 time.
Word 52 ("matter") appears 1 time.
Word 53 ("michael") appears 1 time.
Word 54 ("mind") appears 1 time.
Word 62 ("person") appears 1 time.
Word 68 ("psycholog") appears 1 time.
Word 71 ("reason") appears 1 time.
Word 74 ("relationship") appears 1 time.
Word 78 ("say") appears 1 time.
Word 82 ("show") appears 1 time.
Word 84 ("sound") appears 1 time.
Word 92 ("troubl") appears 1 time.
Word 100 ("act") appears 1 time.
Word 104 ("appear") appears 1 time.
Word 105 ("basic") appears 1 time.
W

In [ ]:
#run LDA using bag of words
#Because LDA uses randomness in both training and inference steps, setting a random state is very important to control the randomness to make answers repeatable.
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2,random_state=2023)

In [ ]:
#for each topic, we will explore the words occuring in that topic and its relative weight.

for idx, topic in lda_model.print_topics(-1): # print all the topics. (-1) means all the topics.
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.009*"relationship" + 0.009*"murder" + 0.007*"chang" + 0.007*"actor" + 0.007*"best" + 0.007*"plot" + 0.007*"woman" + 0.006*"reason" + 0.006*"perform" + 0.006*"long"
Topic: 1 
Words: 0.007*"feel" + 0.007*"real" + 0.007*"better" + 0.007*"citi" + 0.006*"person" + 0.006*"talk" + 0.006*"world" + 0.006*"audienc" + 0.006*"see" + 0.006*"question"
Topic: 2 
Words: 0.012*"paul" + 0.011*"wife" + 0.009*"tri" + 0.008*"music" + 0.007*"robert" + 0.007*"show" + 0.007*"comedi" + 0.006*"place" + 0.006*"audienc" + 0.006*"end"
Topic: 3 
Words: 0.008*"world" + 0.008*"role" + 0.007*"feel" + 0.006*"see" + 0.006*"wife" + 0.006*"screen" + 0.006*"reason" + 0.005*"comedi" + 0.005*"away" + 0.005*"book"
Topic: 4 
Words: 0.011*"music" + 0.011*"releas" + 0.011*"littl" + 0.008*"action" + 0.008*"moment" + 0.007*"perform" + 0.007*"audienc" + 0.007*"emot" + 0.007*"produc" + 0.006*"point"
Topic: 5 
Words: 0.011*"crime" + 0.010*"artist" + 0.009*"polic" + 0.009*"tri" + 0.008*"place" + 0.008*"american" + 0

In [ ]:
lda_model[bow_corpus[101]]

[(8, 0.9955868)]

In [ ]:
topic_coverage = [lda_model[doc] for doc in bow_corpus]

for i, doc_topics in enumerate(topic_coverage):
    print(f"Document {i}:")
    for topic_id, prob in doc_topics:
        print(f"  Topic {topic_id}: {prob:.4f}")

Document 0:
  Topic 1: 0.4365
  Topic 7: 0.5572
Document 1:
  Topic 6: 0.8579
  Topic 7: 0.1352
Document 2:
  Topic 3: 0.7865
  Topic 6: 0.2099
Document 3:
  Topic 6: 0.8468
  Topic 8: 0.1481
Document 4:
  Topic 6: 0.2852
  Topic 8: 0.7090
Document 5:
  Topic 7: 0.9958
Document 6:
  Topic 6: 0.1513
  Topic 7: 0.3823
  Topic 8: 0.4601
Document 7:
  Topic 2: 0.2241
  Topic 5: 0.7466
  Topic 7: 0.0221
Document 8:
  Topic 6: 0.0784
  Topic 7: 0.2549
  Topic 8: 0.6633
Document 9:
  Topic 6: 0.9939
Document 10:
  Topic 6: 0.9212
  Topic 7: 0.0689
Document 11:
  Topic 7: 0.9903
Document 12:
  Topic 0: 0.2309
  Topic 4: 0.3463
  Topic 5: 0.3006
  Topic 6: 0.1190
Document 13:
  Topic 5: 0.9887
Document 14:
  Topic 5: 0.1983
  Topic 6: 0.0580
  Topic 7: 0.7392
Document 15:
  Topic 2: 0.8289
  Topic 6: 0.1654
Document 16:
  Topic 7: 0.9913
Document 17:
  Topic 4: 0.0120
  Topic 7: 0.9783
Document 18:
  Topic 6: 0.0858
  Topic 7: 0.8999
Document 19:
  Topic 5: 0.0144
  Topic 6: 0.9771
Document 20: